# RSNA-Validation Inferences

In this notebook, we test the performance of the baseline model with the validation dataset. We will look at the accuracy, precision and recall for all six labels.

In [1]:
import os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from albumentations import Compose, ShiftScaleRotate, Resize, CenterCrop, HorizontalFlip, RandomBrightnessContrast
from albumentations.pytorch import ToTensor

In [2]:
import tensorflow as tf

import torch
import torch.optim as optim
from torch.utils.data import Dataset

In [3]:
img_dir = "stage_1_train_images_npy"

In [4]:
valid_table = pd.read_pickle('rsna_valid_table.pkl') 

## Create dataset, transform, and dataloader

In [5]:
class RSNA_dataset(Dataset):
    def __init__(self, img_dir, data_table, transform=None):
        
        self.img_dir = img_dir
        self.data_table = data_table
        self.transform = transform

    def __len__(self):
        
        return len(self.data_table)

    def __getitem__(self, idx):
                
        img_file = os.path.join(img_dir,"ID_"+self.data_table.index[idx]+".npy")
        img = np.load(img_file)   
        
        if self.transform:       
                  
            img = np.dstack([img]*3)
            augmented = self.transform(image=img)
            img = augmented['image']   
            
        return {'image': img, 'id' : self.data_table.index[idx]}
        

In [6]:
transform_valid= Compose([Resize(200, 200),
                         ToTensor()
                        ])

In [7]:
valid_data = RSNA_dataset(img_dir, valid_table, transform = transform_valid)

In [8]:
dataloaders = torch.utils.data.DataLoader(valid_data, batch_size = 1,
                                             shuffle=False, num_workers = 1)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
attributes = valid_table.columns.drop('count')

In [10]:
for x in attributes:
    pred_x = 'pred_'+x
    valid_table[pred_x] = np.nan

In [11]:
valid_table

,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,any,count,pred_epidural,pred_intraparenchymal,pred_intraventricular,pred_subarachnoid,pred_subdural,pred_any
case id,,,,,,,,,,,,,
00178eb80,1,0,0,0,0,1,1,NaN,NaN,NaN,NaN,NaN,NaN
001bb2c00,1,0,0,0,0,1,1,NaN,NaN,NaN,NaN,NaN,NaN
004966e2d,1,0,0,0,0,1,1,NaN,NaN,NaN,NaN,NaN,NaN
0237b835f,1,0,0,0,0,1,1,NaN,NaN,NaN,NaN,NaN,NaN
02e2d273d,1,0,0,0,0,1,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
fffcf9972,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
fffe0dae1,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
ffff225b3,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Model

device = torch.device("cuda:0")
model = torch.hub.load('facebookresearch/WSL-Images', 'resnext101_32x8d_wsl')
model.fc = torch.nn.Linear(2048, 6)

model.load_state_dict(torch.load("resnet_three_epoches.pth"))

model.eval()

model.to(device)


Using cache found in /root/.cache/torch/hub/facebookresearch_WSL-Images_master


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

In [13]:
# Iterate over data.

#test_pred = pd.DataFrame(columns = ['ID','Label'])

for i, inputs in enumerate(dataloaders):
    
    img = inputs['image']
    case_id = inputs['id'][0]
         
    img = img.to(device, dtype=torch.float)
    
    with torch.set_grad_enabled(False):
        outputs = model(img)    
        pred = torch.sigmoid(outputs).cpu().numpy().reshape(6,)
        
        valid_table.loc[case_id,'pred_any'] = pred[0]
        valid_table.loc[case_id,'pred_epidural'] = pred[1]
        valid_table.loc[case_id,'pred_intraparenchymal'] = pred[2]
        valid_table.loc[case_id,'pred_intraventricular'] = pred[3]
        valid_table.loc[case_id,'pred_subarachnoid'] = pred[4]
        valid_table.loc[case_id,'pred_subdural'] = pred[5]

In [16]:
valid_table.to_pickle('rsna_validation_inference.pkl')

In [17]:
valid_table

,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,any,count,pred_epidural,pred_intraparenchymal,pred_intraventricular,pred_subarachnoid,pred_subdural,pred_any
case id,,,,,,,,,,,,,
00178eb80,1,0,0,0,0,1,1,0.128519,0.034292,0.004985,0.201209,0.533903,0.697726
001bb2c00,1,0,0,0,0,1,1,0.026601,0.006027,0.000326,0.008425,0.711826,0.747261
004966e2d,1,0,0,0,0,1,1,0.143278,0.064228,0.002981,0.081887,0.234275,0.352317
0237b835f,1,0,0,0,0,1,1,0.093044,0.077247,0.001230,0.048942,0.358961,0.479050
02e2d273d,1,0,0,0,0,1,1,0.020153,0.671732,0.101890,0.124641,0.115613,0.811772
...,...,...,...,...,...,...,...,...,...,...,...,...,...
fffcf9972,0,0,0,0,0,0,0,0.000027,0.000035,0.000015,0.000558,0.000180,0.000666
fffe0dae1,0,0,0,0,0,0,0,0.009112,0.021634,0.002452,0.269274,0.233895,0.397450
ffff225b3,0,0,0,0,0,0,0,0.000241,0.001494,0.000242,0.002213,0.003109,0.006742


In [18]:
for label in attributes:
    
    # Accuracy: the number of correct predictions divided by the number of images
    accuracy = (valid_table['pred_'+label].apply(round) == valid_table[label]).sum()/len(valid_table)

    a = valid_table['pred_'+label].apply(round) == 1
    b = valid_table[label] == 1

    # Precision: of all the images predicted to have this 'label', what percentage actually has that label
    precision = (a & b).sum()/a.sum()

    # Recall: of all the images that have this 'label', what percentage was predicted to have this label
    recall = (a & b).sum()/b.sum()

    print('*** '+label+' ***\n')
    print('accuracy: '+str(accuracy)+'\n')
    print('precision: '+str(precision)+'\n')
    print('recall: '+str(recall)+'\n')
    
#Also look at the precision and recall for normal cases    
a = valid_table['pred_any'].apply(round) == 0
b = valid_table['any'] == 0
precision = (a & b).sum()/a.sum()
recall = (a & b).sum()/b.sum()
print('*** normal ***\n')
print('precision: '+str(precision)+'\n')
print('recall: '+str(recall)+'\n')    

*** epidural ***

accuracy: 0.996600588212826

precision: 0.48878923766816146

recall: 0.31054131054131057

*** intraparenchymal ***

accuracy: 0.9881975478400367

precision: 0.8852835136020977

recall: 0.7529969333705046

*** intraventricular ***

accuracy: 0.9920839540124518

precision: 0.8309178743961353

recall: 0.8346138293570562

*** subarachnoid ***

accuracy: 0.9797563118291891

precision: 0.7662863452337136

recall: 0.5836837678721615

*** subdural ***

accuracy: 0.9703219892288301

precision: 0.739409984871407

recall: 0.6932624113475178

*** any ***

accuracy: 0.9591306672777968

precision: 0.858229313052823

recall: 0.8018846064725419

*** normal ***

precision: 0.9723082902393471

recall: 0.981313400961025

